# 注意力基础知识
在此 notebook 中，我们将了解如何实施注意力机制。我们将重点讨论如何单独实施注意力机制，而不是采用更大型的模型。因为在现实模型中实施注意力机制时，很多精力都放在了数据管道和调整各种参数上，而不是注意力概念本身。

我们将实施注意力评分方法并计算注意力语境向量。

## 注意力评分函数
### 评分函数的输入
首先看看我们将提供给评分函数的输入。我们假设位于解码阶段的第一步。评分函数的第一个输入是解码器的隐藏状态（假设是一个有三个隐藏节点的小型 RNN，不适合现实应用，但是易于描绘）：

In [ ]:
dec_hidden_state = [5,1,20]

我们可视化该向量：

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Let's visualize our decoder hidden state
plt.figure(figsize=(1.5, 4.5))
sns.heatmap(np.transpose(np.matrix(dec_hidden_state)), annot=True, cmap=sns.light_palette("purple", as_cmap=True), linewidths=1)

第一个评分函数将对单个注释（编码器隐藏状态）进行评分，如下所示：

In [ ]:
annotation = [3,12,45] #e.g. Encoder hidden state

In [ ]:
# Let's visualize the single annotation
plt.figure(figsize=(1.5, 4.5))
sns.heatmap(np.transpose(np.matrix(annotation)), annot=True, cmap=sns.light_palette("orange", as_cmap=True), linewidths=1)

### 实现：对单个注释评分
我们计算下单个注释的点积。Numpy 的 [dot()](https://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html) 很适合这一运算

In [ ]:
def single_dot_attention_score(dec_hidden_state, enc_hidden_state):
    # TODO: return the dot product of the two vectors
    return 
    
single_dot_attention_score(dec_hidden_state, annotation)

### 注释矩阵
我们现在看看如何一次性对所有注释评分。为此，看看我们的注释矩阵：

In [ ]:
annotations = np.transpose([[3,12,45], [59,2,5], [1,43,5], [4,3,45.3]])

可以如下所示地可视化（每列是一个编码器时间步的隐藏状态）：

In [ ]:
# Let's visualize our annotation (each column is an annotation)
ax = sns.heatmap(annotations, annot=True, cmap=sns.light_palette("orange", as_cmap=True), linewidths=1)

### 实现：同时对所有注释评分
我们使用矩阵乘法一次性对所有注释评分。继续使用点积评分方法

<img src="images/scoring_functions.png" />

为此，我们需要转置 `dec_hidden_state` 并与 `annotations` 进行[矩阵乘法](https://docs.scipy.org/doc/numpy/reference/generated/numpy.matmul.html)运算。

In [ ]:
def dot_attention_score(dec_hidden_state, annotations):
    # TODO: return the product of dec_hidden_state transpose and enc_hidden_states
    return 
    
attention_weights_raw = dot_attention_score(dec_hidden_state, annotations)
attention_weights_raw

看看这些得分，能看出在此时间步，四个向量中哪个向量从解码器那获得的注意力最高吗？

## Softmax
算出得分后，我们应用 softmax：
<img src="images/softmax.png" />

In [ ]:
def softmax(x):
    x = np.array(x, dtype=np.float128)
    e_x = np.exp(x)
    return e_x / e_x.sum(axis=0) 

attention_weights = softmax(attention_weights_raw)
attention_weights

即使知道哪个注释获得的注意力最多，但是看到 softmax 使最终得分变化如此之大，还是很有趣。第一个和最后一个注释的得分分别为 927 和 929，但是应用 softmax 后，它们获得的注意力分别为 0.12 和 0.88。

# 重新对注释应用得分
算出得分后，我们将每个注释与其得分相乘，以便更接近注意力语境向量。下面是该公式的乘法部分（我们将在后面的单元格中处理加法部分）

<img src="images/Context_vector.png" />

In [ ]:
def apply_attention_scores(attention_weights, annotations):
    # TODO: Multiple the annotations by their weights
    return

applied_attention = apply_attention_scores(attention_weights, annotations)
applied_attention

现在已经重新应用注意力得分，我们来可视化语境向量：

In [ ]:
# Let's visualize our annotations after applying attention to them
ax = sns.heatmap(applied_attention, annot=True, cmap=sns.light_palette("orange", as_cmap=True), linewidths=1)

将结果与之前在 notebook 中可视化的原始注释进行对比，可以看出第二个和第三个注释（列）几乎被消除了。第一个注释保留一定的值，第四个注释比重最大。

# 计算注意力语境向量
生成注意力语境向量的最后一步是将四列求和，生成单个注意力语境向量

In [ ]:
def calculate_attention_vector(applied_attention):
    return np.sum(applied_attention, axis=1)

attention_vector = calculate_attention_vector(applied_attention)
attention_vector

In [ ]:
# Let's visualize the attention context vector
plt.figure(figsize=(1.5, 4.5))
sns.heatmap(np.transpose(np.matrix(attention_vector)), annot=True, cmap=sns.light_palette("Blue", as_cmap=True), linewidths=1)

获得语境向量后，我们可以将其与隐藏状态相连，并传入隐藏层，生成此解码时间步的结果。